# Topic modeling

In [4]:
# !pip install -qU pip wheel
# !pip install -qU gensim tqdm pandas nltk numpy
# import nltk
# nltk.download('stopwords')

In [5]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import gensim
import random
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
TOTAL_TOPICS = 25

In [7]:
# corpus
path_corpus=os.path.expanduser('~/ppa_data/solrcorpus')
path_metadata = os.path.join(path_corpus, 'metadata.csv')
path_minimal = os.path.join(path_corpus, 'minimal.jsonl')
path_texts = os.path.join(path_corpus, 'texts')
path_minimal_numlines=None

In [8]:
# Read metadata
df_metadata = pd.read_csv(path_metadata).fillna('')
df_metadata

,work_id,source_id,group_id_s,source_t,source_url,title,sort_title,pub_date,pub_place,publisher,...,cluster_id_s,item_type,order,work_type_s,last_modified,_version_,first_page_i,book_journal_s,subtitle,notes
0,dul1.ark:|13960|t6d23816n,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6...,"Lyrical ballads, with other poems: in two volu...","Lyrical ballads, with other poems: in two volu...",1802.0,Philadelphia,James Humphreys,...,dul1.ark:/13960/t6d23816n,work,0,full-work,2023-02-02T15:00:45.957Z,1756731849797795847,,,,
1,dul1.ark:|13960|t6d23816n-p11,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n-p11,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6...,Preface [to Lyrical Ballads],Preface [to Lyrical Ballads],1802.0,Philadelphia,James Humphreys,...,dul1.ark:/13960/t6d23816n-p11,work,0,excerpt,2023-02-14T19:59:25.780Z,1757837803736006658,11.0,"Lyrical ballads, with other poems: in two volu...",,
2,inu.32000006890968,inu.32000006890968,inu.32000006890968,['HathiTrust'],https://hdl.handle.net/2027/inu.32000006890968,"The mystical poets of the English church,","mystical poets of the English church,",1919.0,London,Society for Promoting Christian Knowledge,...,osmondmystical,work,0,full-work,2023-02-14T19:34:42.875Z,1757836248763858945,,,,
3,hvd.hn5bic,hvd.hn5bic,hvd.hn5bic,['HathiTrust'],https://hdl.handle.net/2027/hvd.hn5bic,An analysis of the derivative words in the Eng...,analysis of the derivative words in the Englis...,1841.0,New York,Clement & Packard,...,townanalysis,work,0,full-work,2023-02-14T19:59:31.530Z,1757837809744347137,,,"or, A key to their precise analytic definition...",
4,CW0100721039,CW0100721039,CW0100721039,['Gale'],https://link.gale.com/apps/doc/CW0100721039/EC...,A new and general biographical dictionary,new and general biographical dictionary contai...,1798.0,London,"G.G. and J. Robinson, J. Johnson, J. Nichols, ...",...,CW0100721039,work,0,full-work,2023-02-14T19:34:42.875Z,1757836248776441856,,,containing an historical and critical account ...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,njp.32101076530979,njp.32101076530979,njp.32101076530979,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,"Essays philosophical and moral, historical and...","Essays philosophical and moral, historical and...",1799.0,London,G.G. and J. Robinson,...,njp.32101076530979,work,0,full-work,2023-02-02T15:00:44.887Z,1756731848685256707,,,,
6935,njp.32101076530979-p464,njp.32101076530979,njp.32101076530979-p476,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,On Shakespeare,On Shakespeare,1799.0,London,G.G. and J. Robinson,...,njp.32101076530979-p476,work,0,excerpt,2023-02-14T19:59:23.288Z,1757837801082060801,476.0,"Essays philosophical and moral, historical and...",,
6936,njp.32101076530979-p482,njp.32101076530979,njp.32101076530979-p494,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,On Stile and Versification,On Stile and Versification,1799.0,London,G.G. and J. Robinson,...,njp.32101076530979-p494,work,0,excerpt,2023-02-14T19:59:23.288Z,1757837801083109376,494.0,"Essays philosophical and moral, historical and...",,
6937,njp.32101076530979-p514,njp.32101076530979,njp.32101076530979-p526,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,On Epic Poetry,On Epic Poetry,1799.0,London,G.G. and J. Robinson,...,njp.32101076530979-p526,work,0,excerpt,2023-02-14T19:59:23.288Z,1757837801079963654,526.0,"Essays philosophical and moral, historical and...",,


In [61]:
import nltk
from nltk.corpus import stopwords as stops
stopwords = set(stops.words('english')) | {'one','may'}
# stopwords

In [62]:
import orjson

In [63]:
import jsonlines,orjson

def iter_id_tokens(min_toks=50):
    global path_minimal_numlines
    if path_minimal_numlines == None:
        with open(path_minimal) as f:
            path_minimal_numlines = sum(1 for line in tqdm(f,desc='Getting number of lines',position=0))

    with open(path_minimal) as f:
        for ln in tqdm(f,total=path_minimal_numlines,desc='Iterating over jsonl',position=0):
            d=orjson.loads(ln)
            yield (d['id'], d['toks'])
    
def iter_tokens():
    for id,toks in iter_id_tokens():
        yield toks

In [66]:
# iter = iter_id_tokens()
# next(iter)
all_page_ids = {id for id,toks in iter_id_tokens()}
len(all_page_ids)

Iterating over jsonl: 100%|██████████| 2097745/2097745 [01:34<00:00, 22295.15it/s]


1982342

In [67]:
# next(iter) 

# Transforming corpus into bag of words vectors

We can now perform feature engineering by leveraging a simple Bag of Words
model.

In [68]:
fn=os.path.join(path_corpus,'data.gensim.dictionary.pkl')
if not os.path.exists(fn):
    dictionary = gensim.corpora.Dictionary(iter_tokens())
    dictionary.save(fn)
else:
    dictionary = gensim.corpora.Dictionary.load(fn)
len(dictionary)

2023-11-29 09:51:53,498 : INFO : loading Dictionary object from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.dictionary.pkl
2023-11-29 09:51:55,060 : INFO : Dictionary lifecycle event {'fname': '/Users/ryanheuser/ppa_data/solrcorpus/data.gensim.dictionary.pkl', 'datetime': '2023-11-29T09:51:55.060602', 'gensim': '4.3.2', 'python': '3.12.0 (main, Nov  7 2023, 18:55:06) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'loaded'}


2027040

In [69]:
dictionary.filter_extremes(no_above=.9, no_below=10)
# dictionary.filter_extremes(keep_n=50000)
len(dictionary)

2023-11-29 09:51:59,997 : INFO : discarding 1927040 tokens: [('the', 1992801), ('womn', 89), ('and', 1940820), ('atreet', 24), ('elpjjia', 1), ('genium', 130), ('in', 1951311), ('of', 1978646), ('papiniane', 3), ('14j', 41)]...
2023-11-29 09:51:59,998 : INFO : keeping 100000 tokens which were in no less than 10 and no more than 1887970 (=90.0%) documents
2023-11-29 09:52:00,723 : INFO : resulting dictionary: Dictionary<100000 unique tokens: ["'r", 'bennett', 'coleridge', 'collection', 'duke']...>


100000

In [70]:
# Transforming corpus into bag of words vectors
SLICE_LENGTH = 1000
from collections import Counter

def iter_corpus(slice_length=SLICE_LENGTH,max_pages_per_work=100,min_words_per_page=25):
    lastid=None
    toks=[]
    ids=[]
    idnumpages=0
    for i,(page_id,page_toks) in enumerate(iter_id_tokens()):
        id=page_id.split('_')[0]
        
        if not min_words_per_page or len(page_toks) >= min_words_per_page:
            toks.extend([tok for tok in page_toks if tok not in stopwords and len(tok)>2])
            ids.append(page_id)
            idnumpages+=1

        if lastid!=None and id!=lastid:
            # if len(toks)>=slice_length and (not max_pages_per_work or idnumpages<=max_pages_per_work):
            if len(toks)>=slice_length and (not max_pages_per_work or idnumpages<=max_pages_per_work):
                yield (lastid,ids,toks)
            toks = []
            ids = []
            idnumpages = 0
        
        lastid = id

def iter_corpus(max_pages_per_work=25,min_words_per_page=25):
    idnumpages=Counter()
    for i,(page_id,page_toks) in enumerate(iter_id_tokens()):
        page_toks = [tok for tok in page_toks if tok not in stopwords and len(tok)>2]
        id=page_id.split('_')[0]
        if not min_words_per_page or len(page_toks) >= min_words_per_page:
            idnumpages[id]+=1
            if not max_pages_per_work or idnumpages[id]<=max_pages_per_work:
                yield (id,page_id,page_toks)
        

def iter_corpus_tokens(slice_length=SLICE_LENGTH):
    for work_id,page_ids,toks in iter_corpus(slice_length=slice_length):
        yield dictionary.doc2bow(toks)

# next(iter_corpus())
# next(iter_corpus_tokens())

In [71]:
# Transforming corpus into bag of words vectors
def get_gensim_corpus(force=False):
    fn=os.path.join(path_corpus,'data.gensim.corpus.pkl')
    if force or not os.path.exists(fn):
        gensim.corpora.MmCorpus.serialize(
            fname=fn,
            corpus=iter_corpus_tokens(),
            id2word=dictionary,
        )

    corpus = gensim.corpora.MmCorpus(fn)
    return corpus

In [18]:
corpus = get_gensim_corpus(force=True)

2023-11-29 08:55:14,550 : INFO : storing corpus in Matrix Market format to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
2023-11-29 08:55:14,572 : INFO : saving sparse matrix to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
Iterating over jsonl: 100%|██████████| 2097745/2097745 [02:44<00:00, 12725.07it/s]
2023-11-29 08:57:59,428 : INFO : saved 163069x100000 matrix, density=0.110% (17994269/16306900000)
2023-11-29 08:57:59,429 : INFO : saving MmCorpus index to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl.index
2023-11-29 08:57:59,455 : INFO : loaded corpus index from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl.index
2023-11-29 08:57:59,456 : INFO : initializing cython corpus reader from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
2023-11-29 08:57:59,457 : INFO : accepted corpus with 163069 documents, 100000 features, 17994269 non-zero entries


In [32]:
def topic_model(num_topics=TOTAL_TOPICS, sample=1000, force=False):
    fn=os.path.join(path_corpus,f'data.gensim.lda_model.ntopic={num_topics}.pkl')
    corpus_sample = corpus

    if force or not os.path.exists(fn):
        lda_model = gensim.models.ldamulticore.LdaMulticore(
            corpus=corpus_sample, 
            id2word=dictionary,
            num_topics=num_topics,
            workers=7,
            # chunksize=100,
            # alpha='auto', 
            # eta='auto', 
            random_state=42,
            iterations=50, 
            passes=5, 
            # eval_every=None
        )
        lda_model.save(fn)
    else:
        lda_model = gensim.models.LdaModel.load(fn)
    return lda_model

In [33]:
lda_model = topic_model(100)

2023-11-29 09:14:37,436 : INFO : using symmetric alpha at 0.01
2023-11-29 09:14:37,437 : INFO : using symmetric eta at 0.01
2023-11-29 09:14:37,457 : INFO : using serial LDA version on this node
2023-11-29 09:14:38,720 : INFO : running online LDA training, 100 topics, 5 passes over the supplied corpus of 163069 documents, updating every 14000 documents, evaluating every ~140000 documents, iterating 50x with a convergence threshold of 0.001000
2023-11-29 09:14:38,721 : INFO : training LDA model using 7 processes
2023-11-29 09:14:49,508 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/163069, outstanding queue size 1
2023-11-29 09:14:49,827 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/163069, outstanding queue size 2
2023-11-29 09:14:50,090 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/163069, outstanding queue size 3
2023-11-29 09:14:50,355 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/1630

In [34]:
topics_coherences = lda_model.top_topics(corpus, topn=20)
avg_coherence_score = np.mean([item[1] for item in topics_coherences])
print('Avg. Coherence Score:', avg_coherence_score)

2023-11-29 09:34:27,384 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2023-11-29 09:34:27,489 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2023-11-29 09:34:27,594 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2023-11-29 09:34:27,699 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2023-11-29 09:34:27,805 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2023-11-29 09:34:27,913 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2023-11-29 09:34:28,010 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2023-11-29 09:34:28,125 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2023-11-29 09:34:28,238 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2023-11-29 09:34:28,337 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2023-11-29 09:34:28,451 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2023-11-29 09:34:28

Avg. Coherence Score: -2.9346454377173643


Topic coherence is a complex topic in its own and it can be used to measure the
quality of topic models to some extent. Typically, a set of statements is said to be
coherent if they support each other. Topic models are unsupervised learning based
models that are trained on unstructured text data, making it difficult to measure the
quality of outputs.

Refer to Text Analytics with Python 2nd Edition for more detail on this.

In [37]:
lda_model.num_topics

100

In [38]:
def get_topic_terms(lda_model, topn=100):
    return {
        topicid:', '.join([dictionary[i] for i,j in lda_model.get_topic_terms(topicid,topn=topn)])
        for topicid in range(lda_model.num_topics)
    }

In [39]:
get_topic_terms(lda_model)

{0: 'every, would, much, great, upon, without, rules, art, must, manner, ought, well, nature, language, many, though, time, proper, general, genius, indeed, read, others, make, therefore, even, good, cannot, first, knowledge, different, attention, give, part, propriety, either, study, yet, among, fame, made, public, necessary, order, however, kind, learned, use, several, composition, writing, regard, method, perhaps, point, men, particular, far, often, able, natural, never, whether, might, writers, learning, reason, find, persons, speak, found, man, degree, way, advantage, words, little, variety, view, want, could, ever, long, less, whole, best, reading, thing, useful, always, another, agreeable, means, capable, greatest, parts, former, true, care, nothing',
 1: 'first, greek, ancient, learning, man, scaliger, says, son, among, latin, much, account, god, bentley, combe, age, death, latter, though, author, would, editions, also, planus, writers, part, tardus, called, velox, great, find,

In [44]:
data = lda_model.get_document_topics(corpus)

In [72]:
# done_ids = {page_id for docid,page_id,toks in iter_corpus()}

Iterating over jsonl: 100%|██████████| 2097745/2097745 [03:12<00:00, 10887.70it/s]


In [74]:
# undone_ids = all_page_ids - done_ids
# # undone_ids

In [107]:
# doctopics=[]
# for docid,pageid,toks in iter_corpus(None,None):
#     if pageid in undone_ids:
#         bow = dictionary.doc2bow(toks)
#         docs = lda_model.get_document_topics(bow)
#         index,vals = zip(*docs)
#         sdocs = pd.Series(vals, index=index).sort_values(ascending=False)
#         docs2 = lda_model.inference([bow])
#         sdocs2 = pd.Series(docs2[0][0]).sort_values(ascending=False)
#         break

Iterating over jsonl:   0%|          | 28/2097745 [00:00<11:07, 3144.32it/s]


In [108]:
sdocs.sum()

0.96467066

In [118]:
sdocs2.sum()

26.886023

In [121]:
sdocs2/sdocs2.sum()

15    0.535725
16    0.137418
68    0.125203
40    0.085632
95    0.080688
        ...   
32    0.000372
31    0.000372
30    0.000372
29    0.000372
99    0.000372
Length: 100, dtype: float32

In [123]:
sdocdf=pd.DataFrame({'m1':sdocs, 'm2':sdocs2/sdocs2.sum()}).sort_values('m1',ascending=False)
sdocdf['diff'] = sdocdf['m1'] - sdocdf['m2']
sdocdf.head(25)

,m1,m2,diff
15,0.542084,0.535725,0.006359
16,0.138032,0.137418,0.000614
68,0.115294,0.125203,-0.009909
40,0.088471,0.085632,0.002840
95,0.080790,0.080688,0.000101
0,NaN,0.000372,NaN
1,NaN,0.000372,NaN
2,NaN,0.000372,NaN
3,NaN,0.000372,NaN
4,NaN,0.000372,NaN


In [88]:
s=pd.Series(docs2[0][0])

In [94]:
s.sort_values(ascending=False)

15    14.719552
16     3.723455
68     2.924183
40     2.397353
95     2.176149
        ...    
32     0.010000
31     0.010000
30     0.010000
29     0.010000
99     0.010000
Length: 100, dtype: float32